# Tweeteval sentiment detection

In [1]:
import pandas as pd
from pathlib import Path
from scipy.stats import entropy
from collections import Counter

In [2]:
!git clone https://github.com/cardiffnlp/tweeteval.git

Cloning into 'tweeteval'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 370 (delta 13), reused 3 (delta 1), pack-reused 354
Receiving objects: 100% (370/370), 8.49 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [3]:
DATA_DIR = Path("tweeteval") / "datasets"
SENTIMENT_DIR = DATA_DIR / "sentiment"
GPT_INPUT_PATH = Path("gpt-input")

GPT_INPUT_PATH.mkdir(exist_ok=True)

In [4]:
MAPPING = {
    "0":	"negative",
    "1":	"neutral",
    "2":	"positive"

}

RESULTS = ["0", "1", "2"]

In [24]:
df_text = pd.read_csv(SENTIMENT_DIR / "test_text.txt", sep="delimiter", header=None, names=["text"])
df_labels = pd.read_csv(SENTIMENT_DIR / "test_labels.txt", sep="delimiter", header=None, names=["labels"])

df_text["labels"] = df_labels["labels"]

df_text.reset_index(inplace=True, drop=False)

# Split the DataFrame
selected = pd.concat([df_text.iloc[:1115], df_text.iloc[2000:3160],df_text.iloc[4000:5111],df_text.iloc[6000:6888],df_text.iloc[8000:8015],df_text.iloc[10000:10854]])
not_selected = pd.concat([df_text.iloc[1115:2000],df_text.iloc[3160:4000], df_text.iloc[5111:6000], df_text.iloc[6888:8000], df_text.iloc[8015:10000],df_text.iloc[10854:]])

# Create a dictionary to hold random samples for each label
random_examples_per_label = {}

# Group by label and sample 10 random examples for each label in not_selected
for label, group in not_selected.groupby('labels'):
    random_examples_per_label[label] = group.sample(n=10, random_state=1) if len(group) >= 10 else group

#print (random_examples_per_label)

<ipython-input-24-1f31dc7020f3>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_text = pd.read_csv(SENTIMENT_DIR / "test_text.txt", sep="delimiter", header=None, names=["text"])
<ipython-input-24-1f31dc7020f3>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_labels = pd.read_csv(SENTIMENT_DIR / "test_labels.txt", sep="delimiter", header=None, names=["labels"])


In [25]:
def get_query(row, shots):
  text=row['text']
  pattern = f"Knowing that a certain person described the sentiment of the following tweets with one among {MAPPING}:\n\n"
  for lab in range(3):
    for ex in range(shots):
      pattern = pattern + 'tweet:'+random_examples_per_label[lab].iloc[ex]['text'] + ' sentiment:'+str(random_examples_per_label[lab].iloc[ex]['labels'])+'\n\n'
  pattern = pattern + f"\nDescribe the sentiment of the given tweet. Choose your answer from {MAPPING} and return an integer as a result. Do not explain yourself. \n\n"+f"Tweet: {text}."
  return pattern

In [27]:
entropy(list(Counter(selected.labels).values())) # dataset entropy

1.0337578264709495

In [26]:
def generate_dataframe(df: pd.DataFrame):
  for shots in range(2,11,2):
      column_name = f'prompt{shots}'
      df[column_name] = df.apply(get_query, axis=1, args=(shots,))
  return df

In [28]:
new_df = generate_dataframe(selected)
new_df.to_csv(GPT_INPUT_PATH / "22_sentiment_prompts_few_shots.csv", index=False)